In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re


In [24]:
with open("the-verdict.txt" , 'r') as file:
    data = file.read()
# data = input("Enter your prompt here: ")

In [3]:
print(len(data))

18


In [4]:
split_text = re.split(r'([,.<>/!@#$%^&*():;]|--|\s)',data)

In [5]:
#This a for loop for separating the text. And below is list comprehension

# result = []
# for item in text:
#     if item.strip():
#         result.append(item)
    
# result

In [6]:

# This is a word based tokenization algorithm. This tokenize for each word.....

result = [item.strip() for item in split_text if item.strip()]
print(result)

['Hi', 'I', 'am', 'Don', 'Christ']


In [7]:
len(result)

5

In [8]:
words = sorted(set(result))
print(len(words))

5


In [9]:
token_id = {token:i for i,token in enumerate(words)}
print(token_id)

{'Christ': 0, 'Don': 1, 'Hi': 2, 'I': 3, 'am': 4}


In [15]:
import importlib
import tiktoken

print("tik version = ", importlib.metadata.version("tiktoken"))

tik version =  0.12.0


In [13]:
tokenizer = tiktoken.get_encoding("gpt2")

In [14]:
text = ("Helllo, do you like tea? <|endoftext|> In the sunlight terraces"
        "of someUnknownPlace.")

interger = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(interger)

[28254, 5439, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 19606, 8812, 2114, 1659, 617, 20035, 27271, 13]


In [16]:
strings = tokenizer.decode(interger)
print(strings)

Helllo, do you like tea? <|endoftext|> In the sunlight terracesof someUnknownPlace.


In [17]:
# input-target pair
with open("the-verdict.txt" , 'r') as file:
    data = file.read()

In [19]:
enc_text = tokenizer.encode(data)
print(len(enc_text))

5145


In [21]:
sample_context = 20
for i in range(1, sample_context+1):
    context = enc_text[:i]
    desire = enc_text[i]

    print(tokenizer.decode(context), "------->", tokenizer.decode([desire]))

I ------->  H
I H -------> AD
I HAD ------->  always
I HAD always ------->  thought
I HAD always thought ------->  Jack
I HAD always thought Jack ------->  G
I HAD always thought Jack G -------> is
I HAD always thought Jack Gis -------> burn
I HAD always thought Jack Gisburn ------->  rather
I HAD always thought Jack Gisburn rather ------->  a
I HAD always thought Jack Gisburn rather a ------->  cheap
I HAD always thought Jack Gisburn rather a cheap ------->  genius
I HAD always thought Jack Gisburn rather a cheap genius -------> --
I HAD always thought Jack Gisburn rather a cheap genius-- -------> though
I HAD always thought Jack Gisburn rather a cheap genius--though ------->  a
I HAD always thought Jack Gisburn rather a cheap genius--though a ------->  good
I HAD always thought Jack Gisburn rather a cheap genius--though a good ------->  fellow
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow ------->  enough
I HAD always thought Jack Gisburn rather a chea

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_len, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
    # Using sliding window to chuck book into overlapping sequence of max_len 
        for i in range (0, len(token_ids)-max_len, stride):
            input_chuck = token_ids[i : i + max_len]
            target_chuck = token_ids[i + 1 : i + max_len + 1]
            self.input_ids.append(torch.tensor(input_chuck))
            self.target_ids.append(torch.tensor(target_chuck))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    

def create_dataloader(txt, batch_size = 4, max_len = 256, stride = 128, shuffle = True, drop_last = True, num_workers = 0):
    dataset = GPTDatasetV1(txt, tokenizer, max_len, stride) #create dataset
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle = shuffle, drop_last = drop_last, num_workers = num_workers)
    return dataloader


#with open("the-verdict.txt" , 'r') as file:
#    data = file.read()      Run this to get the data


dataloader = create_dataloader(data, batch_size=1, max_len=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

# [tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])] ----->> it gives input_tensor and output_tensor

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
